In [1]:
!pip install pyspark

In [ ]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [7]:
dataframe_site = spark.read.json('site.json')

AnalysisException: 'Path does not exist: file:/C:/Users/ricar/git/curso_big_data_com_mapreduce_spark/08_NLP/site.json;'

In [ ]:
dataframe_site

In [ ]:
dataframe_loja = spark.read.format('orc').load('loja.orc')

In [ ]:
dataframe_loja

In [ ]:
dataframe_contabilidade = spark.read.format('csv').option('header', True).option("delimiter", ";").load("contabilidade.csv.gz")

In [ ]:
dataframe_contabilidade

In [ ]:
dataframe_site.show()

In [ ]:
dataframe_loja.show()

In [ ]:
dataframe_contabilidade.show()

In [ ]:
dataframe_site.printSchema()

In [ ]:
dataframe_loja.printSchema()

In [ ]:
dataframe_contabilidade.printSchema()

In [ ]:
from pyspark.sql.types import *

In [ ]:
schema = StructType([StructField("id", IntegerType(), True),
                     StructField("status", StringType(), True),
                     StructField("data_pedido", TimestampType(), True)])

In [ ]:
dataframe_contabilidade = spark.read.format('csv').schema(schema).option('header', True).option("delimiter", ";").load("contabilidade.csv.gz")

In [ ]:
dataframe_contabilidade.printSchema()

In [ ]:
dataframe_inner = dataframe_contabilidade.join(dataframe_site, dataframe_contabilidade.id == dataframe_site.id_ped)

In [ ]:
dataframe_inner.count()

In [ ]:
dataframe_inner.show()

In [ ]:
dataframe_site_loja = dataframe_loja.join(dataframe_site, dataframe_loja.id_pedido == dataframe_site.id_ped, how='full')

In [ ]:
dataframe_site_loja.show()

In [ ]:
dataframe_site_loja.count()

In [ ]:
dataframe_site_loja_dis = dataframe_site_loja.distinct()

In [ ]:
dataframe_site_loja_dis.count()

In [ ]:
dataframe_contabilidade.filter(dataframe_contabilidade.status == 'ENT').show(30, False)

In [ ]:
dataframe_fevereiro = dataframe_contabilidade.filter(dataframe_contabilidade.status == 'ENT')

In [ ]:
dataframe_fevereiro

In [ ]:
dataframe_fevereiro.filter(dataframe_fevereiro.data_pedido >= '2019-02-01').filter(dataframe_fevereiro.data_pedido <= '2019-02-28').show()

In [ ]:
dataframe_fevereiro = dataframe_fevereiro.filter(dataframe_fevereiro.data_pedido >= '2019-02-01').filter(dataframe_fevereiro.data_pedido <= '2019-02-28')

In [ ]:
dataframe_fevereiro.show()

In [ ]:
dataframe_fevereiro.count()

In [ ]:
dataframe_site_loja

In [ ]:
dataframe_groupby = dataframe_site_loja.groupBy(dataframe_site_loja.id_pedido).agg(F.sum(dataframe_site_loja.preco_venda).alias('preco_venda_agrupado'))

In [ ]:
import pyspark.sql.functions as F

In [ ]:
dataframe_groupby.count()

In [ ]:
dataframe_site_loja.count()

In [ ]:
dataframe_groupby.show()

In [ ]:
dataframe_coalesce = dataframe_site_loja.withColumn("nova_coluna", coalesce(dataframe_site_loja.preco_venda, dataframe_site_loja.frete_valor))

In [ ]:
dataframe_coalesce.select(dataframe_coalesce.preco_venda, dataframe_coalesce.frete_valor, dataframe_coalesce.nova_coluna).show()

In [ ]:
dataframe_coalesce.where(dataframe_coalesce.nova_coluna.isNull()).count()

In [ ]:
dataframe_fevereiro.crossJoin(dataframe_fevereiro.withColumnRenamed('id', 'id2')).show()

In [ ]:
dataframe_crossjoin = dataframe_fevereiro.crossJoin(dataframe_fevereiro.withColumnRenamed('id', 'id2'))

In [ ]:
dataframe_crossjoin.describe

In [ ]:
dataframe_crossjoin.drop(dataframe_crossjoin.id2)

In [ ]:
dataframe_crossjoin.printSchema()

In [ ]:
dataframe_exclusao =  dataframe_crossjoin.drop(dataframe_crossjoin.id2)

In [ ]:
dataframe_exclusao.printSchema()

In [ ]:
dataframe_exclusao_linhas =  dataframe_fevereiro.dropDuplicates()

In [ ]:
dataframe_exclusao_linhas.count()

In [ ]:
dataframe_fevereiro.count()

In [ ]:
dataframe_novo = spark.createDataFrame([
    ("ENT", "ENTREGA"),
    ("RET", "RETIRA"),
    ("TRN", "ATRASO"),
    ("EST", "ESTOQUE")],
    ["id_entrega_status", "descricao"])

In [ ]:
dataframe_novo.describe

In [ ]:
dataframe_novo.first()

In [ ]:
dataframe_descricao = dataframe_contabilidade.join(dataframe_novo, dataframe_contabilidade.status == dataframe_novo.id_entrega_status)

In [ ]:
dataframe_descricao.show()

In [ ]:
dataframe_pandas = dataframe_descricao.toPandas()

In [ ]:
dataframe_json = dataframe_fevereiro.toJSON()

In [ ]:
type(dataframe_json)

In [ ]:
dataframe_json

In [ ]:
type(dataframe_fevereiro)

In [ ]:
dataframe_union = dataframe_fevereiro.union(dataframe_exclusao_linhas)

In [ ]:
dataframe_fevereiro.count()

In [ ]:
dataframe_exclusao_linhas.count()

In [ ]:
dataframe_union.count()

In [ ]:
dataframe_union.limit(10).show()

In [ ]:
from pyspark.sql.functions import col

In [ ]:
dataframe_site_loja = dataframe_site_loja.sort(col('preco_venda').desc())

In [ ]:
dataframe_site_loja

In [ ]:
dataframe_site_loja.show()

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

In [ ]:
udf_funcao = udf(lambda x : x.replace(".", ","), StringType())

In [ ]:
dataframe_funcao_replace = dataframe_site_loja.select(dataframe_site_loja.id_pedido, dataframe_site_loja.data_entrega, udf_funcao(dataframe_site_loja.frete_valor).alias('frete_valor'), udf_funcao(dataframe_site_loja.preco_venda).alias('preco_venda'), dataframe_site_loja.quantidade, dataframe_site_loja.id_unidade, dataframe_site_loja.id_ped)

In [ ]:
dataframe_site_loja

In [ ]:
dataframe_funcao_replace.limit(5).show()

In [ ]:
from pyspark.sql.functions import split
split_coluna = split(dataframe_funcao_replace['id_ped'], "-")
dataframe_funcao_split = dataframe_funcao_replace.withColumn("coluna_split_0", split_coluna.getItem(0))
dataframe_funcao_split = dataframe_funcao_split.withColumn("coluna_split_1", split_coluna.getItem(1))
dataframe_funcao_split.limit(3).show()

In [ ]:
dataframe_funcao_split = dataframe_funcao_split.drop(dataframe_funcao_split.coluna_split_0)
dataframe_funcao_split.limit(1).show()

In [ ]:
dataframe_descricao.show()

In [ ]:
from pyspark.sql.functions import substring
dataframe_funcao_sub = dataframe_descricao.select(dataframe_descricao.id, dataframe_descricao.status,  dataframe_descricao.descricao, dataframe_descricao.data_pedido, substring(dataframe_descricao["id_entrega_status"], 0,1).alias("id_entrega"))
dataframe_funcao_sub.show()

In [ ]:
dataframe_descricao

In [ ]:
rdd_unidade_venda = spark.sparkContext.textFile("UNIDADES.txt")

In [ ]:
rdd_unidade_venda.collect()

In [ ]:
type(rdd_unidade_venda)

In [ ]:
rdd_unidade_venda_formatado = rdd_unidade_venda.map(lambda x: (x.replace("L", "LOJA").split(",")))

In [ ]:
rdd_unidade_venda_formatado.collect()

In [ ]:
dataframe_unidade = rdd_unidade_venda_formatado.toDF(["codigo", "descricao_codigo"])

In [ ]:
dataframe_unidade.show()

In [ ]:
type(dataframe_unidade)

In [ ]:
dataframe_final = dataframe_funcao_split.join(dataframe_unidade, dataframe_funcao_split.id_unidade == dataframe_unidade.codigo)

In [ ]:
dataframe_final.limit(1).show()

In [ ]:
dataframe_final.write.json('entrega_pedidos')

In [ ]:
dataframe_final.repartition(1).write.json('entrega')

In [ ]:
dataframe_final.repartition(1).write.format('orc').option("encondig", "UTF-8").save("pedidos_entregues")

In [ ]:
dataframe_final_total = dataframe_final.join(dataframe_funcao_sub, dataframe_final.id_pedido == dataframe_funcao_sub.id)

In [ ]:
dataframe_final_total.describe

In [ ]:
dataframe_final_total.repartition(1).write.format('csv').mode("append").option("encondig", "UTF-8").option("header", True).option("delimiter", ";").option("inferSchema", "true").option("compression", "gzip").save("/home/alura/Desktop/pedidos_entregues_completo")